# Transformación a Parquet

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
import glob

In [2]:
spk = SparkSession.builder \
    .config("spark.driver.port", "4040") \
    .master("local[*]") \
    .appName("EDAReview") \
    .getOrCreate()

24/11/01 07:27:14 WARN Utils: Your hostname, hugo-81we resolves to a loopback address: 127.0.1.1; using 10.203.144.90 instead (on interface wlp0s20f3)
24/11/01 07:27:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/01 07:27:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/01 07:27:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/01 07:27:26 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


## Carga de datos

In [3]:
file = glob.glob('reviews-estados/**/*.json')

In [ ]:
df = spk.read.json(file)

In [8]:
df.printSchema()

root
 |-- gmap_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- pics: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- url: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |-- rating: long (nullable = true)
 |-- resp: struct (nullable = true)
 |    |-- text: string (nullable = true)
 |    |-- time: long (nullable = true)
 |-- text: string (nullable = true)
 |-- time: long (nullable = true)
 |-- user_id: string (nullable = true)



In [9]:
df.count()

89946359

In [ ]:
df.filter(df['pics'].isNotNull()).show(5)

In [ ]:
df = df.drop('pics')
df.show(5)

In [ ]:
from pyspark.sql import functions as F

# Convertir la columna de milisegundos a fecha
df = df.withColumn('fecha', F.from_unixtime(F.col('time') / 1000).cast('timestamp'))

# Mostrar el DataFrame actualizado
df.show(5)

In [ ]:
df = df.withColumn('text', F.lower(F.col('text')))

In [ ]:
df = df.withColumn('año', F.year(F.col('fecha')))
df.show(5)

In [ ]:
df.filter(df['user_id'].isNull()).count()

In [ ]:
df.filter(df['time'].isNull()).count()

In [ ]:
df.filter(df['text'].isNull()).count()

In [ ]:
df = df.withColumn('id_rev', F.concat(F.col('user_id'),F.lit('_'), F.col('time')))


In [ ]:
df_duplicados = df.groupBy('id_rev').count().filter(F.col('count') > 1)


In [ ]:
# Obtener una muestra de 5 datos de la columna 'id_rev' del df de duplicados
muestras = df_duplicados.select("id_rev").rdd.map(lambda row: row[0]).takeSample(False, 5)

In [ ]:
# Filtrar el DataFrame
df_muestras = df.filter(df['id_rev'].isin(muestras))

# Mostrar el DataFrame filtrado
df_muestras.show()

In [ ]:
#Eliminar Duplicados
df = df.dropDuplicates(["id_rev"])

In [ ]:
df.write.option("compression", "snappy").parquet('review')